In [4]:
import json
from tqdm import tqdm
import os
import googlemaps
with open("data.json") as f:
    data = json.load(f)
print(len(data))

6036


In [2]:
data[2]

{'id': 2116846707,
 'date_published': '2009-10-06',
 'citation_count': 437,
 'estimated_citation_count': 1191,
 'title': 'Suspending Damage: A Letter to Communities',
 'authors': [{'DAuN': 'Eve Tuck',
   'AuId': 2165230311,
   'DAfN': 'State University of New York System',
   'AfId': 1327163397}],
 'fields': [{'DFN': 'Oppression', 'FId': 2776526686},
  {'DFN': 'Social exclusion', 'FId': 134659438},
  {'DFN': 'Accountability', 'FId': 2776007630},
  {'DFN': 'Educational research', 'FId': 36727532},
  {'DFN': 'Disadvantaged', 'FId': 2780623907},
  {'DFN': 'Environmental ethics', 'FId': 95124753},
  {'DFN': 'Theory of change', 'FId': 7340896},
  {'DFN': 'Law', 'FId': 199539241},
  {'DFN': 'Conflict resolution', 'FId': 21711469},
  {'DFN': 'Sociology', 'FId': 144024400},
  {'DFN': 'Perspective-taking', 'FId': 2780342009}],
 'journal': {'JN': 'harvard educational review', 'JId': 129458801},
 'DOI': '10.17763/HAER.79.3.N0016675661T3N15',
 'source': [{'U': 'http://www.hepgjournals.org/doi/abs/

In [3]:
paper_lookup = {}
for node in data:
    paper_lookup[node["id"]] = node

In [8]:

filename = "institutions.json"
if os.path.isfile(filename):
    with open("institutions.json", "r") as f:
        institutions = json.load(f)
else:
    institutions = {}
    for node in data:
        seen_inst_this_node = []
        for a in node.get("authors"):
            id = a.get("AfId")
            if id and id not in seen_inst_this_node:
                seen_inst_this_node.append(id)
                if id not in institutions:
                    institutions[id] = {
                        "id": id,
                        "name": a.get("DAfN"),
                        "n_papers": 0,
                        "n_citations": 0,
                        "links": {}
                    }
                institutions[id]["n_papers"] += 1
                institutions[id]["n_citations"] += node["citation_count"]
                for ref in node["references"]:
                    other = paper_lookup[ref]
                    for oa in other.get("authors"):
                        oid = oa.get("AfId")
                        if oid:
                            if oid not in institutions[id]["links"]:
                                institutions[id]["links"][oid] = 0
                            institutions[id]["links"][oid] += 1
    institutions = list(institutions.values())
    print(len(institutions))
    gmaps = googlemaps.Client(key='INSERT_KEY_HERE')
    for inst in tqdm(institutions):
        if "geo" not in inst:
            inst["geo"] = gmaps.geocode(inst["name"])
    with open("institutions.json", "w") as f:
        json.dump(institutions, f)

print(len(institutions))

966


In [13]:
for inst in institutions:
    for node in data:
        date = node.get("date_published")
        for a in node.get("authors"):
            if a.get("DAfN") == inst["name"]:
                if not inst.get("date_published") or inst.get("date_published") > date:
                    inst["date_published"] = date
print(institutions[0])

{'id': 1327163397, 'name': 'State University of New York System', 'n_papers': 3, 'n_citations': 435, 'links': {'174216632': 3, '223532165': 1}, 'geo': [{'address_components': [{'long_name': '353', 'short_name': '353', 'types': ['street_number']}, {'long_name': 'Broadway', 'short_name': 'Broadway', 'types': ['route']}, {'long_name': 'Downtown', 'short_name': 'Downtown', 'types': ['neighborhood', 'political']}, {'long_name': 'Albany', 'short_name': 'Albany', 'types': ['locality', 'political']}, {'long_name': 'Albany County', 'short_name': 'Albany County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'New York', 'short_name': 'NY', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '12207', 'short_name': '12207', 'types': ['postal_code']}], 'formatted_address': '353 Broadway, Albany, NY 12207, USA', 'geometry': {'location': {'lat': 42.6480051, 'lng': -73.74948

In [14]:
with open("institutions.json", "w") as f:
    json.dump(institutions, f)
!ls -lah institutions.json

-rw-r--r-- 1 nyou045 nyou045 1.4M Dec  4 15:33 institutions.json
